# 메타데이터에 모델 점수를 연결

In [ ]:
import pandas as pd
import pickle
import numpy as np
from pathlib import Path

base_path = Path('./experiment/matches_scores')
models = [model.name for model in list(base_path.glob('*'))]

with open('./Dataset/meta_analysis.ftr', 'rb') as file:
    meta = pd.read_feather(file)

meta.insert(5, 'gold', (meta['gold_earned']+meta['gold_spent']))
meta.drop(['gold_earned', 'gold_spent'], axis='columns', inplace=True)

for model in models:
    meta[model] = np.nan
    model_list = list((base_path / model).glob('*'))
    for idx, match in enumerate(model_list):
        endstr = '\n' if idx+1 == len(model_list) else '\r'
        print(f'[{model}] ({idx+1:5d}/{len(model_list)}) {(idx+1)/len(model_list)*100:3.1f}% progressing...', end=endstr)

        with match.open('rb') as file: scores = pickle.load(file)
        scores = [scores[idx].sum().item() for idx in range(10)]

        meta.loc[(meta.mat_no==match.name[:-4]), model] = scores

meta.to_feather('./Dataset/meta_analysis_with_score.ftr')
meta

# 각 매치별 플레이어 KDA, Gold, Creep score, 모델 점수를 순위화한 데이터 생성

In [12]:
import pandas as pd
import pickle
import numpy as np
from tqdm import tqdm

meta = pd.read_feather('./Dataset/meta_analysis_with_score.ftr')
meta.drop('teamid', axis='columns', inplace=True)
columns = meta.columns[4:]

block_num = int(len(meta) / 10)
ar = np.zeros([len(meta), len(columns)], dtype=np.int8)
df = pd.DataFrame(ar, columns=columns)
df.insert(0, 'match', np.nan)
df.insert(1, 'player', np.nan)

for idx in tqdm(range(block_num)):
    sdx = idx * 10
    edx = sdx + 9
    tmp = meta.loc[sdx:edx].copy()
    df.loc[sdx:edx, 'match'] = tmp['mat_no']
    df.loc[sdx:edx, 'player'] = tmp['player']+1

    for col in columns:
        tmp.sort_values(by=col, axis=0, inplace=True, ignore_index=True, ascending=False)
        data = [int(tmp.index[tmp.player==pi].item()+1) for pi in range(10)]
        df.loc[sdx:edx, col] = data

tqdm.pandas()
df['avg'] = df.progress_apply((lambda x: int((x.gold+x.creep+x.kda)/3)), axis=1)
df['avg_nocreep'] = df.progress_apply((lambda x: int((x.gold+x.kda)/2)), axis=1)

df.to_feather('./experiment/analysis_result.ftr')
df

100%|██████████| 455750/455750 [00:03<00:00, 122853.90it/s]


,match,player,kda,gold,creep,win,normal,normal_bce,mlp_bce,zero_h0_bce,zero_h0_reverse,zero_h0,mlp,avg,avg_nocreep
0,210513_na1_3902040674,0.0,10,10,6,8,10,9,7,10,9,10,10,8,10
1,210513_na1_3902040674,1.0,6,7,7,9,8,8,10,7,4,5,5,6,6
2,210513_na1_3902040674,2.0,9,9,5,7,6,10,9,9,10,9,9,7,9
3,210513_na1_3902040674,3.0,7,6,4,6,9,7,5,6,7,8,8,5,6
4,210513_na1_3902040674,4.0,8,8,8,10,7,6,8,8,8,7,7,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455745,210517_jp1_302513885,5.0,2,2,5,3,4,1,3,7,5,5,3,3,2
455746,210517_jp1_302513885,6.0,1,3,10,5,2,3,2,1,1,2,1,4,2
455747,210517_jp1_302513885,7.0,3,1,1,1,1,4,1,2,3,1,2,1,2
455748,210517_jp1_302513885,8.0,6,4,3,2,3,5,6,3,2,3,8,4,5
